In [14]:
import numpy as np
import open3d as o3d
from pyslam import config, io, slam, dense

def run_slam_on_video(video_path, cam_calib_path, output_map_path=None):
    """
    Run pySLAM full SLAM (with dense mapping) on a video,
    return camera trajectory and dense point cloud.
    """
    # 1. set up configuration
    cfg = config.Config("config.yaml")
    # override dataset to VIDEO
    cfg["DATASET"]["type"] = "VIDEO_DATASET"
    cfg["DATASET"]["VIDEO_DATASET"]["path"] = video_path
    cfg["DATASET"]["VIDEO_DATASET"]["camera_settings"] = cam_calib_path
    # enable volumetric / dense mapping
    cfg["GLOBAL_PARAMETERS"]["kUseVolumetricIntegration"] = True
    # pick TSDF or Gaussian splatting method
    cfg["GLOBAL_PARAMETERS"]["kVolumetricIntegrationType"] = "TSDF"
    # (optionally) enable depth prediction (if using monocular)
    cfg["GLOBAL_PARAMETERS"]["kUseDepthEstimatorInFrontEnd"] = True

    # 2. run SLAM
    slam_system = slam.SLAM(cfg)
    slam_system.run()  # blocks until finishes (or user quits)

    # 3. extract dense map / point cloud
    dense_integrator = slam_system.get_dense_integrator()
    # get a point cloud (Open3D format) or numpy
    pcd = dense_integrator.get_point_cloud()

    if output_map_path is not None:
        o3d.io.write_point_cloud(output_map_path, pcd)
    return slam_system, pcd

def estimate_scene_bbox(pcd: o3d.geometry.PointCloud):
    """
    Given a point cloud, compute axis-aligned bounding box (or oriented bounding box),
    return length, width, height.
    """
    # Option A: axis-aligned bounding box
    aabb = pcd.get_axis_aligned_bounding_box()
    dims = aabb.get_extent()  # (x_size, y_size, z_size)
    return dims, aabb

def main():
    video_path = "input_scene.mp4"
    cam_calib = "settings/my_camera.yaml"
    output_map = "dense_map.ply"

    slam_sys, pcd = run_slam_on_video(video_path, cam_calib, output_map)
    dims, bbox = estimate_scene_bbox(pcd)
    print("Scene bounding box dims (units):", dims)

    # If you know one real dimension (e.g. width of door = 0.9 m),
    # you can compute scale factor to convert units → meters
    known_scene_distance = 0.9  # meters
    measured_units = dims[0]  # assume x-direction corresponds
    scale = known_scene_distance / measured_units
    dims_meters = dims * scale
    print("Estimated bounding dims in meters:", dims_meters)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'numpy'

In [16]:
import numpy as np
from pyslam import config, slam

def run_slam(video_path: str, calib_path: str) -> np.ndarray:
    """
    Run pySLAM on a video and return a 3D point cloud as a NumPy array.
    (No visualization, no Open3D)
    """
    cfg = config.Config("config.yaml")

    # --- configure dataset ---
    cfg["DATASET"]["type"] = "VIDEO_DATASET"
    cfg["DATASET"]["VIDEO_DATASET"]["path"] = video_path
    cfg["DATASET"]["VIDEO_DATASET"]["camera_settings"] = calib_path

    # --- enable dense reconstruction ---
    cfg["GLOBAL_PARAMETERS"]["kUseVolumetricIntegration"] = True
    cfg["GLOBAL_PARAMETERS"]["kVolumetricIntegrationType"] = "TSDF"
    cfg["GLOBAL_PARAMETERS"]["kUseDepthEstimatorInFrontEnd"] = True

    # --- run SLAM ---
    slam_system = slam.SLAM(cfg)
    slam_system.run()

    # --- get point cloud (Nx3 numpy array) ---
    dense_integrator = slam_system.get_dense_integrator()
    points = dense_integrator.get_points_numpy()  # suppose it returns np.ndarray
    return points


def compute_scene_size(points: np.ndarray):
    """
    Compute simple bounding box dimensions (length, width, height)
    from a NumPy point cloud array of shape (N, 3).
    """
    if points is None or len(points) == 0:
        raise ValueError("No points reconstructed from SLAM.")

    # Remove NaNs or infs
    valid = np.isfinite(points).all(axis=1)
    points = points[valid]

    # Compute min/max per axis
    mins = np.min(points, axis=0)
    maxs = np.max(points, axis=0)

    dims = maxs - mins  # [dx, dy, dz]
    return dims, mins, maxs


def scale_to_real_world(dims, known_real_length=None, known_axis=0):
    """
    Optional: rescale dimensions to meters if one known real-world length is given.
    known_axis = 0 (x), 1 (y), or 2 (z)
    """
    if known_real_length is None:
        return dims, 1.0
    scale = known_real_length / dims[known_axis]
    return dims * scale, scale


def main():
    video_path = "input_scene.mp4"
    calib_path = "camera.yaml"  # your calibration file

    print("Running SLAM... (this can take a while)")
    points = run_slam(video_path, calib_path)

    print(f"Reconstructed {len(points)} 3D points.")
    dims, mins, maxs = compute_scene_size(points)

    # Optional: if you know a real object’s size to fix scale
    # e.g., wall width known = 3.2 meters
    dims_m, scale = scale_to_real_world(dims, known_real_length=3.2, known_axis=0)

    print("Bounding box (in SLAM units):", dims)
    print(f"Scale factor: {scale:.3f}")
    print("Estimated scene size (meters):")
    print(f"  Length (x): {dims_m[0]:.2f} m")
    print(f"  Width  (y): {dims_m[1]:.2f} m")
    print(f"  Height (z): {dims_m[2]:.2f} m")


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'numpy'